In [1]:
# ============================================================================
# 0. SETUP AND DATA LOADING
# ============================================================================

import sys, os, warnings
import pandas as pd
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

warnings.filterwarnings('ignore')
os.chdir('game-behavior-analytics/data_analysis_notebook/')
sys.path.append(os.path.abspath('utils'))

from data_utils import load_and_prepare_data
from metadata import theory_order, theory_map

df, concepts = load_and_prepare_data("../data/final_dataset.csv")

print("="*80)
print("RQ3: HOW DO INTERVENTIONS REDUCE CHEATING WITHOUT HARMING PERFORMANCE AND EXPERIENCE?")
print("SYSTEMATIC ANALYSIS OF PSYCHOLOGICAL MECHANISMS")
print("="*80)
print(f"Sample size: {len(df)}\n")



RQ3: HOW DO INTERVENTIONS REDUCE CHEATING WITHOUT HARMING PERFORMANCE AND EXPERIENCE?
SYSTEMATIC ANALYSIS OF PSYCHOLOGICAL MECHANISMS
Sample size: 1232



In [2]:
# ============================================================================
# 1. DATA TYPE CLASSIFICATION AND PREPARATION
# ============================================================================

print("1. DATA TYPE CLASSIFICATION\n" + "-"*40)

# Rename variables for clarity
mechanism_renames = {
    'descriptive_norms': 'perceived_descriptive_norms',
    'injunctive_norms': 'perceived_injunctive_norms',
    'reference_group_identification': 'perceived_group_identification',
    'social_sanctions': 'perceived_social_sanctions',
    'performance_accomplishments': 'perceived_performance_accomplishments',
    'vicarious_experience': 'perceived_vicarious_experience',
    'verbal_persuasion': 'perceived_verbal_persuasion',
    'emotional_arousal': 'perceived_emotional_arousal'
}

pme_renames = {
    'PME_on_honest_task_completion': 'perceived_honesty',
    'PME_on_task_performance': 'perceived_performance_effect',
    'PME_on_task_experience': 'perceived_experience_effect'
}

df.rename(columns={**mechanism_renames, **pme_renames}, inplace=True)

# Handle perceived ability
if 'word_creation_skill_level' in df.columns:
    df['perceived_ability'] = df.pop('word_creation_skill_level')

# Define variables
mechanisms = [
    'autonomy_need_satisfaction', 'autonomy_need_frustration',
    'competence_need_satisfaction', 'competence_need_frustration',
    'relatedness_need_satisfaction', 'relatedness_need_frustration',
    'cognitive_discomfort', 'moral_disengagement',
    'perceived_descriptive_norms', 'perceived_injunctive_norms',
    'perceived_group_identification', 'perceived_social_sanctions',
    'perceived_performance_accomplishments', 'perceived_vicarious_experience',
    'perceived_verbal_persuasion', 'perceived_emotional_arousal',
    'perceived_honesty', 'perceived_performance_effect',
    'perceived_experience_effect', 'perceived_ability'
]

actual_outcomes = ['cheating_behavior', 'performance', 'experience']
continuous_vars = mechanisms + ['performance', 'experience']
data_types = {
    'ordinal': ['cheating_behavior'],
    'categorical': ['concept'],
    'continuous': continuous_vars
}

print(f"Ordinal: {data_types['ordinal']}\nCategorical: {data_types['categorical']}\nContinuous: {len(continuous_vars)} variables\n")




1. DATA TYPE CLASSIFICATION
----------------------------------------
Ordinal: ['cheating_behavior']
Categorical: ['concept']
Continuous: 22 variables



In [3]:
# ============================================================================
# 2. THEORETICAL FRAMEWORK MAPPING
# ============================================================================

print("2. THEORETICAL FRAMEWORK MAPPING")
print("-" * 40)

# Define theoretical frameworks and their mechanisms
theoretical_frameworks = {
    'Self-Determination': {
        'concepts': ['autonomy', 'competence', 'relatedness'],
        'mechanisms': ['autonomy_need_satisfaction', 'autonomy_need_frustration',
                      'competence_need_satisfaction', 'competence_need_frustration',
                      'relatedness_need_satisfaction', 'relatedness_need_frustration']
    },
    'Cognitive-Dissonance': {
        'concepts': ['self_concept', 'cognitive_inconsistency', 'dissonance_arousal', 'dissonance_reduction'],
        'mechanisms': ['cognitive_discomfort', 'moral_disengagement']
    },
    'Self-Efficacy': {
        'concepts': ['performance_accomplishments', 'vicarious_experience', 'verbal_persuasion', 'emotional_arousal'],
        'mechanisms': ['perceived_performance_accomplishments', 'perceived_vicarious_experience', 
                      'perceived_verbal_persuasion', 'perceived_emotional_arousal']
    },
    'Social-Norms': {
        'concepts': ['descriptive_norms', 'injunctive_norms', 'social_sanctions', 'reference_group_identification'],
        'mechanisms': ['perceived_descriptive_norms', 'perceived_injunctive_norms', 
                      'perceived_group_identification', 'perceived_social_sanctions']
    },
    'Perceived-Effectiveness': {
        'concepts': [],  # No direct concepts
        'mechanisms': ['perceived_honesty', 'perceived_performance_effect', 'perceived_experience_effect']
    },
    'Individual-Differences': {
        'concepts': [],
        'mechanisms': ['perceived_ability']
    }
}

# Create concept-to-theory and mechanism-to-theory mappings
concept_to_theory = {}
mechanism_to_theory = {}

for theory, items in theoretical_frameworks.items():
    for concept in items['concepts']:
        concept_to_theory[concept] = theory
    for mechanism in items['mechanisms']:
        mechanism_to_theory[mechanism] = theory

print("Theoretical frameworks defined:")
for theory, items in theoretical_frameworks.items():
    print(f"  {theory}: {len(items['concepts'])} concepts, {len(items['mechanisms'])} mechanisms")
print()


2. THEORETICAL FRAMEWORK MAPPING
----------------------------------------
Theoretical frameworks defined:
  Self-Determination: 3 concepts, 6 mechanisms
  Cognitive-Dissonance: 4 concepts, 2 mechanisms
  Self-Efficacy: 4 concepts, 4 mechanisms
  Social-Norms: 4 concepts, 4 mechanisms
  Perceived-Effectiveness: 0 concepts, 3 mechanisms
  Individual-Differences: 0 concepts, 1 mechanisms



In [11]:
# ============================================================================
# 3. PARTIAL CORRELATION FUNCTION
# ============================================================================

def partial_corr(df, var1, var2, controls=[]):
    """Compute partial correlation controlling for controls"""
    vars_needed = [var1,var2]+controls
    data = df[vars_needed].dropna()
    if len(data)<10: return np.nan
    X = StandardScaler().fit_transform(data[controls]) if controls else None
    y1,y2 = StandardScaler().fit_transform(data[[var1,var2]]).T
    if controls:
        resid1 = y1 - LinearRegression().fit(X,y1).predict(X)
        resid2 = y2 - LinearRegression().fit(X,y2).predict(X)
        return np.corrcoef(resid1,resid2)[0,1] if len(np.unique(resid1))>1 and len(np.unique(resid2))>1 else np.nan
    else:
        return np.corrcoef(y1,y2)[0,1]

In [12]:
# ============================================================================
# 4. CREATE CONCEPT DUMMY VARIABLES
# ============================================================================

print("3. CREATING CONCEPT DUMMY VARIABLES\n" + "-"*40)
all_concepts = [c for d in theoretical_frameworks.values() for c in d['concepts'] if c in df['concept'].unique()]
for c in all_concepts: df[f'concept_{c}'] = (df['concept']==c).astype(int)
concept_dummies = [f'concept_{c}' for c in all_concepts]
print(f"Created {len(concept_dummies)} concept dummy variables\n")


3. CREATING CONCEPT DUMMY VARIABLES
----------------------------------------
Created 15 concept dummy variables



In [13]:
# ============================================================================
# 5. COMPREHENSIVE PARTIAL CORRELATION ANALYSIS
# ============================================================================

def compute_edges(data, group_name="All"):
    print(f"4. COMPREHENSIVE PARTIAL CORRELATION ANALYSIS - {group_name.upper()}\n" + "-"*60)
    available_mechs = [m for m in mechanisms if m in data.columns and not data[m].isna().all()]
    available_outs = [o for o in actual_outcomes if o in data.columns and not data[o].isna().all()]
    available_concepts = [c for c in concept_dummies if c in data.columns]
    edges=[]
    # Concept → Mechanism
    for cd in available_concepts:
        for m in available_mechs:
            ctrl=[c for c in available_concepts if c!=cd]+available_outs
            pc = partial_corr(data, cd, m, ctrl)
            if not np.isnan(pc): edges.append({'source':cd.replace('concept_',''),'target':m,'partial_correlation':pc,'edge_type':'concept_to_mechanism','group':group_name})
    # Mechanism ↔ Mechanism
    for m1,m2 in combinations(available_mechs,2):
        ctrl=available_concepts+available_outs
        pc=partial_corr(data,m1,m2,ctrl)
        if not np.isnan(pc): edges.append({'source':m1,'target':m2,'partial_correlation':pc,'edge_type':'mechanism_to_mechanism','group':group_name})
    # Mechanism → Outcome
    for m in available_mechs:
        for o in available_outs:
            if m==o: continue
            ctrl=available_concepts+[oo for oo in available_outs if oo!=o]
            pc=partial_corr(data,m,o,ctrl)
            if not np.isnan(pc): edges.append({'source':m,'target':o,'partial_correlation':pc,'edge_type':'mechanism_to_outcome','group':group_name})
    # Outcome ↔ Outcome
    for o1,o2 in combinations(available_outs,2):
        ctrl=available_concepts+available_mechs
        pc=partial_corr(data,o1,o2,ctrl)
        if not np.isnan(pc): edges.append({'source':o1,'target':o2,'partial_correlation':pc,'edge_type':'outcome_to_outcome','group':group_name})
    print(f"Total edges calculated: {len(edges)}")
    return pd.DataFrame(edges)

In [14]:
# ============================================================================
# 6. CALCULATE PARTIAL CORRELATIONS FOR ALL GROUPS
# ============================================================================

all_edges_df = compute_edges(df,"All")
cheater_groups={'non_cheaters':0,'partial_cheaters':1,'full_cheaters':2}
group_edges=[]
for gname,val in cheater_groups.items():
    grp=df[df['cheating_behavior']==val]
    if len(grp)>20: group_edges.append(compute_edges(grp,gname))

combined_edges = pd.concat([all_edges_df]+group_edges,ignore_index=True) if group_edges else all_edges_df
print(f"\nCOMBINED RESULTS SUMMARY:\nTotal edges: {len(combined_edges)}\nEdges by type:\n{combined_edges.groupby(['edge_type','group']).size()}")
combined_edges.to_csv('network_plot_partial_correlations.csv',index=False)
print(f"\nSaved to: network_plot_partial_correlations.csv\n")
print()

4. COMPREHENSIVE PARTIAL CORRELATION ANALYSIS - ALL
------------------------------------------------------------
Total edges calculated: 553
4. COMPREHENSIVE PARTIAL CORRELATION ANALYSIS - NON_CHEATERS
------------------------------------------------------------
Total edges calculated: 531
4. COMPREHENSIVE PARTIAL CORRELATION ANALYSIS - PARTIAL_CHEATERS
------------------------------------------------------------
Total edges calculated: 531
4. COMPREHENSIVE PARTIAL CORRELATION ANALYSIS - FULL_CHEATERS
------------------------------------------------------------
Total edges calculated: 531

COMBINED RESULTS SUMMARY:
Total edges: 2146
Edges by type:
edge_type               group           
concept_to_mechanism    All                 300
                        full_cheaters       300
                        non_cheaters        300
                        partial_cheaters    300
mechanism_to_mechanism  All                 190
                        full_cheaters       190
               

In [15]:
# ============================================================================
# 7. RQ3.1: DO INTERVENTIONS ACTIVATE THEIR INTENDED MECHANISMS?
# ============================================================================

print("5. RQ3.1: DO INTERVENTIONS ACTIVATE THEIR INTENDED MECHANISMS?")
print("=" * 70)

# Focus on concept-to-mechanism edges for overall sample
concept_mechanism_edges = all_edges_df[all_edges_df['edge_type'] == 'concept_to_mechanism'].copy()

# Define expected relationships (corrected for cognitive dissonance)
expected_relationships = {
    # Self-Determination Theory
    'autonomy': ['autonomy_need_satisfaction', 'autonomy_need_frustration'],
    'competence': ['competence_need_satisfaction', 'competence_need_frustration'], 
    'relatedness': ['relatedness_need_satisfaction', 'relatedness_need_frustration'],
    
    # Cognitive Dissonance Theory - ONLY cognitive_discomfort as specified
    'self_concept': ['cognitive_discomfort'],
    'cognitive_inconsistency': ['cognitive_discomfort'],
    'dissonance_arousal': ['cognitive_discomfort'], 
    'dissonance_reduction': ['cognitive_discomfort'],
    
    # Self-Efficacy Theory
    'performance_accomplishments': ['performance_accomplishments'],
    'vicarious_experience': ['vicarious_experience'],
    'verbal_persuasion': ['verbal_persuasion'],
    'emotional_arousal': ['emotional_arousal'],
    
    # Social Norms Theory
    'descriptive_norms': ['descriptive_norms'],
    'injunctive_norms': ['injunctive_norms'],
    'social_sanctions': ['social_sanctions'],
    'reference_group_identification': ['reference_group_identification']
}

# Analyze expected vs actual relationships
mechanism_activation_results = []

for concept, expected_mechanisms in expected_relationships.items():
    concept_edges = concept_mechanism_edges[concept_mechanism_edges['source'] == concept]
    
    if len(concept_edges) == 0:
        continue
    
    print(f"\n{concept.upper().replace('_', ' ')} CONCEPT:")
    print("-" * 40)
    
    # Check each expected mechanism
    for mechanism in expected_mechanisms:
        matching_edge = concept_edges[concept_edges['target'] == mechanism]
        
        if len(matching_edge) > 0:
            pcorr = matching_edge.iloc[0]['partial_correlation']
            activated = abs(pcorr) > 0.1  # Threshold for meaningful activation
            
            print(f"  → {mechanism}: r = {pcorr:.3f} {'✓ ACTIVATED' if activated else '✗ weak'}")
            
            mechanism_activation_results.append({
                'concept': concept,
                'theory': concept_to_theory.get(concept, 'Unknown'),
                'mechanism': mechanism,
                'partial_correlation': pcorr,
                'activated': activated,
                'expected': True
            })
        else:
            print(f"  → {mechanism}: NO DATA")
    
    # Show unexpected strong activations
    unexpected_strong = concept_edges[
        (~concept_edges['target'].isin(expected_mechanisms)) & 
        (concept_edges['partial_correlation'].abs() > 0.1)
    ].sort_values('partial_correlation', key=abs, ascending=False)
    
    if len(unexpected_strong) > 0:
        print("  Unexpected strong activations:")
        for _, edge in unexpected_strong.head(3).iterrows():
            print(f"    → {edge['target']}: r = {edge['partial_correlation']:.3f}")

# Save mechanism activation results
activation_df = pd.DataFrame(mechanism_activation_results)
if len(activation_df) > 0:
    activation_df.to_csv('mechanism_activation_analysis.csv', index=False)
    
    print(f"\nMECHANISM ACTIVATION SUMMARY:")
    print("-" * 40)
    activation_summary = activation_df.groupby(['theory', 'activated']).size().unstack(fill_value=0)
    print(activation_summary)
    
    overall_activation_rate = activation_df['activated'].mean()
    print(f"\nOverall activation rate: {overall_activation_rate:.1%}")

print()

5. RQ3.1: DO INTERVENTIONS ACTIVATE THEIR INTENDED MECHANISMS?

AUTONOMY CONCEPT:
----------------------------------------
  → autonomy_need_satisfaction: r = 0.025 ✗ weak
  → autonomy_need_frustration: r = -0.014 ✗ weak

COMPETENCE CONCEPT:
----------------------------------------
  → competence_need_satisfaction: r = 0.059 ✗ weak
  → competence_need_frustration: r = -0.091 ✗ weak
  Unexpected strong activations:
    → perceived_emotional_arousal: r = 0.108

RELATEDNESS CONCEPT:
----------------------------------------
  → relatedness_need_satisfaction: r = 0.020 ✗ weak
  → relatedness_need_frustration: r = -0.010 ✗ weak

SELF CONCEPT CONCEPT:
----------------------------------------
  → cognitive_discomfort: r = -0.055 ✗ weak

COGNITIVE INCONSISTENCY CONCEPT:
----------------------------------------
  → cognitive_discomfort: r = 0.001 ✗ weak

DISSONANCE AROUSAL CONCEPT:
----------------------------------------
  → cognitive_discomfort: r = 0.011 ✗ weak

DISSONANCE REDUCTION CONCEPT:


In [ ]:
# ============================================================================
# 8. RQ3.2: MECHANISM INTERCONNECTIONS
# ============================================================================

def analyze_interconnections(edges_df, group="All", show_top=10):
    """Analyze mechanism interconnections"""
    mech_edges = edges_df[(edges_df['edge_type']=='mechanism_to_mechanism') & (edges_df['group']==group)]
    
    # Filter to theoretical mechanisms only
    theory_mechs = sum([v['mechanisms'] for k,v in theoretical_frameworks.items() 
                       if k not in ['Perceived-Effectiveness','Individual-Differences']],[])
    mech_edges = mech_edges[(mech_edges['source'].isin(theory_mechs)) & 
                           (mech_edges['target'].isin(theory_mechs))]
    
    # Classify connections
    mech_edges['source_theory'] = mech_edges['source'].map(mechanism_to_theory)
    mech_edges['target_theory'] = mech_edges['target'].map(mechanism_to_theory)
    mech_edges['conn_type'] = mech_edges.apply(
        lambda r: 'within' if r['source_theory']==r['target_theory'] else 'cross', axis=1)
    
    # Add absolute correlation column for sorting
    mech_edges['abs_corr'] = mech_edges['partial_correlation'].abs()
    
    within = mech_edges[mech_edges['conn_type']=='within'].nlargest(show_top,'abs_corr')
    cross = mech_edges[mech_edges['conn_type']=='cross'].nlargest(show_top,'abs_corr')
    
    return {'within':len(mech_edges[mech_edges['conn_type']=='within']),
            'cross':len(mech_edges[mech_edges['conn_type']=='cross']),
            'within_top':within,'cross_top':cross}

print("6. RQ3.2: HOW DO THEORETICAL MECHANISMS INTERCONNECT?")
print("="*70)

# Overall interconnection analysis
overall_interconn = analyze_interconnections(combined_edges,"All")
print(f"Theoretical mechanism interconnections:")
print(f"  Within-theory: {overall_interconn['within']} (avg |r|: {overall_interconn['within_top']['partial_correlation'].abs().mean():.3f})")
print(f"  Cross-theory: {overall_interconn['cross']} (avg |r|: {overall_interconn['cross_top']['partial_correlation'].abs().mean():.3f})")

print(f"\nSTRONGEST WITHIN-THEORY CONNECTIONS:")
for _,e in overall_interconn['within_top'].iterrows():
    print(f"  {e['source']} ↔ {e['target']}: r={e['partial_correlation']:+.3f} ({e['source_theory']})")

print(f"\nSTRONGEST CROSS-THEORY CONNECTIONS:")
for _,e in overall_interconn['cross_top'].iterrows():
    print(f"  {e['source']} ({e['source_theory']}) ↔ {e['target']} ({e['target_theory']}): r={e['partial_correlation']:+.3f}")

# Group-wise interconnection analysis
print("\n8b. RQ3.2 BY CHEATER GROUP: DIFFERENTIAL INTERCONNECTIONS")
print("="*70)

group_interconn = {}
for group in ['non_cheaters','partial_cheaters','full_cheaters']:
    print(f"\n{group.replace('_',' ').upper()} INTERCONNECTIONS:")
    print("-"*50)
    group_interconn[group] = analyze_interconnections(combined_edges,group,5)
    gi = group_interconn[group]
    within_avg = gi['within_top']['partial_correlation'].abs().mean() if len(gi['within_top'])>0 else 0
    cross_avg = gi['cross_top']['partial_correlation'].abs().mean() if len(gi['cross_top'])>0 else 0
    print(f"Within-theory: {gi['within']} (avg |r|: {within_avg:.3f})")
    print(f"Cross-theory: {gi['cross']} (avg |r|: {cross_avg:.3f})")
    
    cross_ratio = gi['cross']/(gi['within']+gi['cross']) if (gi['within']+gi['cross'])>0 else 0
    print(f"Cross-theory ratio: {cross_ratio:.1%}")
    
    if len(gi['within_top'])>0:
        print("Top within-theory:")
        for _,e in gi['within_top'].head(3).iterrows():
            print(f"  {e['source']} ↔ {e['target']}: r={e['partial_correlation']:+.3f}")
    
    if len(gi['cross_top'])>0:
        print("Top cross-theory:")
        for _,e in gi['cross_top'].head(3).iterrows():
            print(f"  {e['source']} ↔ {e['target']}: r={e['partial_correlation']:+.3f}")

print()

In [ ]:
# ============================================================================
# 9. RQ3.3: OUTCOME PREDICTORS
# ============================================================================

def analyze_predictors(edges_df, group="All", show_top=8):
    """Analyze mechanism-outcome predictors"""
    mech_out = edges_df[(edges_df['edge_type']=='mechanism_to_outcome') & 
                       (edges_df['group']==group) & 
                       (edges_df['target'].isin(['cheating_behavior','performance','experience']))]
    
    predictors = {}
    for outcome in ['cheating_behavior','performance','experience']:
        out_edges = mech_out[mech_out['target']==outcome]
        if len(out_edges)>0:
            out_edges['abs_corr'] = out_edges['partial_correlation'].abs()
            top_pred = out_edges.nlargest(show_top,'abs_corr')
            predictors[outcome] = top_pred
    return predictors

print("7. RQ3.3: WHICH MECHANISMS MOST STRONGLY PREDICT OUTCOMES?")
print("="*70)

# Overall predictor analysis
overall_pred = analyze_predictors(combined_edges,"All")
print("Mechanism → Actual Outcome relationships:")
for outcome,preds in overall_pred.items():
    if len(preds)>0:
        avg_pred_strength = preds['partial_correlation'].abs().mean()
        print(f"\nSTRONGEST PREDICTORS OF {outcome.upper()} (avg |r|: {avg_pred_strength:.3f}):")
        for _,e in preds.iterrows():
            theory = mechanism_to_theory.get(e['source'],'Unknown')
            print(f"  {e['source']}: r={e['partial_correlation']:+.3f} ({theory})")

# Group-wise predictor analysis
print("\n9b. RQ3.3 BY CHEATER GROUP: DIFFERENTIAL PREDICTORS")
print("="*70)

group_pred = {}
for group in ['non_cheaters','partial_cheaters','full_cheaters']:
    print(f"\n{group.replace('_',' ').upper()} PREDICTORS:")
    print("-"*50)
    group_pred[group] = analyze_predictors(combined_edges,group,5)
    
    for outcome,preds in group_pred[group].items():
        if len(preds)>0:
            avg_strength = preds['partial_correlation'].abs().mean()
            print(f"\n{outcome.upper()} predictors (avg |r|: {avg_strength:.3f}):")
            for _,e in preds.head(3).iterrows():
                theory = mechanism_to_theory.get(e['source'],'Unknown')
                print(f"  {e['source']}: r={e['partial_correlation']:+.3f} ({theory})")

print()


In [ ]:
# ============================================================================
# 10. RQ3.4: PERCEIVED-ACTUAL ALIGNMENT
# ============================================================================

def analyze_alignment(edges_df, group="All"):
    """Analyze perceived-actual outcome alignment"""
    perceived_vars = ['perceived_honesty','perceived_performance_effect','perceived_experience_effect']
    actual_vars = ['cheating_behavior','performance','experience']
    
    alignments = []
    for perc in perceived_vars:
        for act in actual_vars:
            # Look for direct edges between perceived and actual
            align_edges = edges_df[(edges_df['group']==group) & 
                                  (edges_df['source']==perc) & 
                                  (edges_df['target']==act)]
            if len(align_edges)>0:
                corr = align_edges.iloc[0]['partial_correlation']
                alignments.append({'perceived':perc,'actual':act,'correlation':corr})
                print(f"  {perc} → {act}: r={corr:.3f}")
    
    return alignments

print("8. RQ3.4: DO PERCEIVED OUTCOMES ALIGN WITH ACTUAL OUTCOMES?")
print("="*70)

# Overall alignment analysis
print("Perceived vs Actual Outcome Correlations:")
overall_align = analyze_alignment(combined_edges,"All")
if overall_align:
    overall_avg_align = np.mean([abs(a['correlation']) for a in overall_align])
    print(f"Overall average perceived-actual alignment: {overall_avg_align:.3f}")
else:
    print("No perceived-actual alignment edges found")

# Group-wise alignment analysis  
print("\n10b. RQ3.4 BY CHEATER GROUP: ALIGNMENT DIFFERENCES")
print("="*70)

group_align = {}
for group in ['non_cheaters','partial_cheaters','full_cheaters']:
    print(f"\n{group.replace('_',' ').upper()} ALIGNMENT:")
    print("-"*45)
    group_align[group] = analyze_alignment(combined_edges,group)
    
    if group_align[group]:
        avg_align = np.mean([abs(a['correlation']) for a in group_align[group]])
        print(f"Average alignment: {avg_align:.3f}")
    else:
        print("No alignment edges found")

print(f"\nALIGNMENT COMPARISON:")
print("-"*40)
for group,aligns in group_align.items():
    if aligns:
        avg = np.mean([abs(a['correlation']) for a in aligns])
        print(f"{group.replace('_',' ').title()}: {avg:.3f}")
    else:
        print(f"{group.replace('_',' ').title()}: No data")

print("\n" + "="*80)
print("RQ3 ANALYSIS COMPLETE")
print("="*80)